In [4]:
# Baseado no tutorial: https://keras.io/examples/nlp/text_classification_from_scratch/

# importando bibliotecas basicas
import tensorflow as tf
import numpy as np
import os
import codecs

from sklearn.model_selection import train_test_split

# Carregando os dados

In [5]:
def load_data(data_dir, class_label):

    file_names = []
    for (dirpath, dirnames, filenames) in os.walk(data_dir):
        for f in filenames:
            if f.endswith(".txt"):
                file_names.append(os.path.join(dirpath, f))

    txt_ = []
    y = []
    for f in file_names: 
        txt_.append(codecs.open(f,'r',encoding='iso8859-1').read())
        y.append(class_label)

    return txt_, y

fake_data, y_fake = load_data('../aula05/Fake.Br Corpus/full_texts/fake/', 1)    
true_data, y_true = load_data('../aula05/Fake.Br Corpus/full_texts/true/', 2)    




In [ ]:
data_train, data_test, y_train, y_test = train_test_split(fake_data + true_data, y_fake + y_true,test_size=0.15, random_state=1447)

In [6]:
# É importante dar uma olhada no seu dado original para garantir a sua normalização
# e tokenização estejam da forma como você espera. Nós podemos fazer isso pegando 
# alguns poucos exemplos do conjunto de treino e olhando eles.
print(data_train[0])
print(y_train[0])

Juiz determina Ã¡rea para manifestantes e proÃ­be acampamento do MST durante julgamento de Lula em Porto Alegre

Recurso do ex-presidente serÃ¡ julgado em segunda instÃ¢ncia em 24 de janeiro de 2018. Pedido do MPF para evitar protestos no dia foi atendido parcialmente pelo juiz.

juiz federal OsÃ³rio Ãvila Neto determinou que seja estabelecida uma Ã¡rea onde os manifestantes deverÃ£o ficar nas imediaÃ§Ãµes do Tribunal Regional Federal da 4a RegiÃ£o, em Porto Alegre, no dia do julgamento do recurso do ex-presidente Luiz InÃ¡cio Lula da Silva em segunda instÃ¢ncia, em 24 de janeiro de 2018. O magistrado ainda citou que deverÃ¡ haver um isolamento para o trÃ¢nsito nas vias prÃ³ximas ao prÃ©dio.

Em maio, durante o primeiro depoimento de Lula como rÃ©u em Curitiba, as autoridades do ParanÃ¡ montaram um esquema de seguranÃ§a em funÃ§Ã£o das manifestaÃ§Ãµes de apoiadores e grupos contrÃ¡rios ao ex-presidente.

A aÃ§Ã£o civil pÃºblica foi protocolada pelo MinistÃ©rio PÃºblico Federal (MPF) e

# Preparando os dados

In [7]:
# bibliotecas necessarias para o pre-processamento de dados
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

In [8]:
# padronizando os nossos dados com o que ja conhecemos: 
# colocando com letra minúscula e removendo pontuacao
def padronizacao_customizada(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(
        lowercase, "[%s]" % re.escape(string.punctuation), ""
    )


# Constantes do modelo
# as nossas features serao nossas palavras, no nosso modelo teremos no maximo 20000 palavras
max_features = 20000 
# aqui a nossa camada de embedding sera embutida, portanto o treinamento dos vetores embeddings acontecerá 
# logo antes de passar pela nossa CNN
embedding_dim = 128 
# tamanho fixo que um texto do nosso dataset pode ter, você pode limitar em qualquer tamanho, mas lembre-se 
# que quanto maior, mais memoria será necessaria
sequence_length = 500

# agora que temos nossa padronização customizada, podemos instanciar o nosso texto
# na camada de vetorização. Nós estamos usando esta camada para normalizar, dividir e mapear
# string para inteiros, assim nós setamos nosso 'output_mode' para 'int'
# Note que nós estamos usando uma função para dividir "padrão", e 
# a normalização definida acima.
# Nós também setamos um tamanho máximo para o nosso texto (sequence_length), visto que
# CNNs em nosso modelo não suportam sequências de tamanhos diferentes.
vectorize_layer = TextVectorization(
    standardize=padronizacao_customizada,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# agora que a camada de vocabulario foi criada, chame 'adapt' em um dataset
# só de texto para criar o vocabulário. Não precisar ser em "lote", mas para pequenos 
# datasets isso significa que você não está mantendo cópias sobressalentes do 
# dataset na memória

# Agora vamos chamar `adapt` para criar o nosso vocabulário:
vectorize_layer.adapt(data_train)

# Vetorizando o texto

In [9]:
# transformacao dos textos em indices de números inteiros
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vetorização dos dados the data.
train_ds,y_train = vectorize_text(data_train, y_train)
test_ds, y_test = vectorize_text(data_test, y_test)

# Construindo um modelo

In [10]:
# aqui vamos usar uma camada de convolução + pooling com uma camada de embedding

from tensorflow.keras import layers

# Uma entrada inteira para os indices do vocabulario
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# A seguir, adicionams uma camada para mapear estes indices em um espaço 
# de dimensionalidade 'embedding_dim'
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# Por fim adicionamos no final uma cama escondida básica:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# E projetammos em uma única unidade de saida 
# para "compactar" com uma sigmoid
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
dropout (Dropout)            (None, None, 128)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         114816    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)              

# Treinamos o modelo

In [16]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(train_ds, np.array(y_train), batch_size=32, epochs=epochs)

Epoch 1/3
192/192 [==============================] - 23s 117ms/step - loss: -793542131712.0000 - accuracy: 0.4993
Epoch 2/3
192/192 [==============================] - 24s 123ms/step - loss: -1297875992576.0000 - accuracy: 0.4993
Epoch 3/3
192/192 [==============================] - 29s 149ms/step - loss: -2010227146752.0000 - accuracy: 0.4993


# Avaliamos o modelo no nosso conjunto de teste

In [14]:
model.evaluate(test_ds, np.array(y_test))

34/34 [==============================] - 1s 20ms/step - loss: -584878325760.0000 - accuracy: 0.5037


[-584878325760.0, 0.5037037134170532]